In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

fn = []

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        fn.append(os.path.join(dirname, filename))
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-top-2000-movies/imdb_top_2000_movies.csv


In [3]:
df = pd.read_csv(fn[0])
df

,Movie Name,Release Year,Duration,IMDB Rating,Metascore,Votes,Genre,Director,Cast,Gross
0,The Godfather,1972,175,9.2,100.0,"2,002,655","Crime, Drama",Francis Ford Coppola,Marlon Brando,$134.97M
1,The Godfather Part II,1974,202,9.0,90.0,"1,358,608","Crime, Drama",Francis Ford Coppola,Al Pacino,$57.30M
2,Ordinary People,1980,124,7.7,86.0,"56,476",Drama,Robert Redford,Donald Sutherland,$54.80M
3,Lawrence of Arabia,1962,218,8.3,100.0,"313,044","Adventure, Biography, Drama",David Lean,Peter O'Toole,$44.82M
4,Straw Dogs,1971,113,7.4,73.0,"64,331","Crime, Drama, Thriller",Sam Peckinpah,Dustin Hoffman,NaN
...,...,...,...,...,...,...,...,...,...,...
1995,The Young Victoria,2009,105,7.2,64.0,"66,235","Biography, Drama, History",Jean-Marc Vallée,Emily Blunt,$11.00M
1996,Tooth Fairy,I 2010,101,5.0,36.0,"49,527","Comedy, Family, Fantasy",Michael Lembeck,Dwayne Johnson,$60.02M
1997,The Informant!,2009,108,6.5,66.0,"67,318","Biography, Comedy, Crime",Steven Soderbergh,Matt Damon,$33.31M
1998,Youth in Revolt,2009,90,6.4,63.0,"75,956","Comedy, Drama, Romance",Miguel Arteta,Michael Cera,$15.28M


In [ ]:
# Convert 'Votes' column to numeric by removing commas
df['Votes'] = df['Votes'].str.replace(',', '').astype(float)

# If 'Gross' or other columns also contain formatting, clean them similarly
# Example for 'Gross': Remove '$' and 'M' (if present) and convert to float
if 'Gross' in df.columns:
    df['Gross'] = df['Gross'].str.replace('[\$,]', '', regex=True).str.replace('M', '', regex=True).astype(float)

In [4]:
df.columns

Index(['Movie Name', 'Release Year', 'Duration', 'IMDB Rating', 'Metascore',
       'Votes', 'Genre', 'Director', 'Cast', 'Gross'],
      dtype='object')

In [5]:
df.iloc[0]

Movie Name             The Godfather
Release Year                    1972
Duration                         175
IMDB Rating                      9.2
Metascore                      100.0
Votes                      2,002,655
Genre                   Crime, Drama
Director        Francis Ford Coppola
Cast                   Marlon Brando
Gross                       $134.97M
Name: 0, dtype: object

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load your dataframe
# Assuming df is already loaded with the mentioned columns

# Fill NaN values in 'Genre' and 'Director' columns to avoid errors
df['Genre'] = df['Genre'].fillna('')
df['Director'] = df['Director'].fillna('')

# Combine relevant features into a single string
df['features'] = df['Genre'] + ' ' + df['Director'] + ' ' + df['Cast']

# Create a TF-IDF matrix
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['features'])

# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to get recommendations
def recommend_movies(movie_name, num_recommendations=5):
    # Check if the movie exists in the dataset
    if movie_name not in df['Movie Name'].values:
        print(f"'{movie_name}' not found in the dataset!")
        return []
    
    # Get the index of the given movie
    movie_idx = df[df['Movie Name'] == movie_name].index[0]
    
    # Get similarity scores for the given movie
    similarity_scores = list(enumerate(cosine_sim[movie_idx]))
    
    # Sort movies by similarity scores
    sorted_movies = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices of the top N similar movies
    recommended_indices = [idx for idx, _ in sorted_movies[1:num_recommendations + 1]]
    
    # Return the names of the recommended movies
    return df['Movie Name'].iloc[recommended_indices].tolist()

# Example usage
movie_to_recommend = 'The Godfather'
recommended_movies = recommend_movies(movie_to_recommend, num_recommendations=5)

print(f"Movies recommended for '{movie_to_recommend}':")
print(recommended_movies)


Movies recommended for 'The Godfather':
['The Godfather Part II', 'The Godfather Part III', 'The Rainmaker', 'The Outsiders', 'Jack']


In [7]:
movie_to_recommend = 'Goodfellas'
recommended_movies = recommend_movies(movie_to_recommend, num_recommendations=5)

print(f"Movies recommended for '{movie_to_recommend}':")
print(recommended_movies)


Movies recommended for 'Goodfellas':
['Taxi Driver', 'Casino', 'The King of Comedy', 'Mean Streets', 'Cape Fear']


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

# Assuming `df` is your DataFrame with the given structure

# Fill NaN values to avoid errors
df['Genre'] = df['Genre'].fillna('')
df['Director'] = df['Director'].fillna('')
df['Cast'] = df['Cast'].fillna('')

# Combine text-based features into a single string
df['text_features'] = df['Genre'] + ' ' + df['Director'] + ' ' + df['Cast']

# Normalize numerical features (IMDB Rating, Duration, etc.)
scaler = MinMaxScaler()
df[['IMDB Rating', 'Duration']] = scaler.fit_transform(df[['IMDB Rating', 'Duration']])

# Vectorize text features using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
text_matrix = tfidf_vectorizer.fit_transform(df['text_features'])

# Combine numerical features with text features
import numpy as np
numerical_features = df[['IMDB Rating', 'Duration']].to_numpy()
combined_features = np.hstack((text_matrix.toarray(), numerical_features))

# Compute cosine similarity
cosine_sim = cosine_similarity(combined_features, combined_features)

# Function to recommend movies
def recommend_movies(movie_name, num_recommendations=5):
    if movie_name not in df['Movie Name'].values:
        print(f"'{movie_name}' not found in the dataset!")
        return []
    
    # Get the index of the given movie
    movie_idx = df[df['Movie Name'] == movie_name].index[0]
    
    # Get similarity scores for the given movie
    similarity_scores = list(enumerate(cosine_sim[movie_idx]))
    
    # Sort movies by similarity scores (excluding itself)
    sorted_movies = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Get indices of the top N similar movies
    recommended_indices = [idx for idx, _ in sorted_movies[1:num_recommendations + 1]]
    
    # Return the names of the recommended movies
    return df['Movie Name'].iloc[recommended_indices].tolist()

# Example usage
movie_to_recommend = 'The Godfather'
recommended_movies = recommend_movies(movie_to_recommend, num_recommendations=5)

print(f"Movies recommended for '{movie_to_recommend}':")
recommended_movies

Movies recommended for 'The Godfather':


['The Godfather Part II',
 'The Godfather Part III',
 'Apocalypse Now',
 'The Rainmaker',
 'On the Waterfront']

In [15]:
from sklearn.cluster import KMeans

# Preprocess numerical features
features = df[['Duration', 'IMDB Rating', 'Metascore']].fillna(0)
kmeans = KMeans(n_clusters=5, random_state=42)
df['Cluster'] = kmeans.fit_predict(features)

def recommend_cluster(movie_name, num_recommendations=5):
    cluster = df[df['Movie Name'] == movie_name]['Cluster'].values[0]
    cluster_movies = df[df['Cluster'] == cluster].sample(num_recommendations)
    return cluster_movies['Movie Name'].tolist()

recommend_cluster('The Godfather')


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


['Hellboy II: The Golden Army',
 'The Fly',
 'The Piano',
 'Sex, Lies, and Videotape',
 'Poltergeist']

In [17]:
def recommend_popular(top_n=5):
    return df.sort_values(by='Votes', ascending=False)['Movie Name'].head(top_n).tolist()

recommend_popular()

['The Birdcage',
 'The Thomas Crown Affair',
 'The International',
 'Holes',
 'I Love You Phillip Morris']

In [19]:
from sklearn.neighbors import NearestNeighbors

# Preprocess numerical features
features = df[['IMDB Rating', 'Metascore', 'Duration']].fillna(0)
nn_model = NearestNeighbors(n_neighbors=6, metric='cosine')
nn_model.fit(features)

def recommend_knn(movie_name, num_recommendations=5):
    idx = df[df['Movie Name'] == movie_name].index[0]
    distances, indices = nn_model.kneighbors(features.iloc[idx].values.reshape(1, -1), n_neighbors=num_recommendations + 1)
    return df.iloc[indices[0][1:]]['Movie Name'].tolist()

recommend_knn('The Godfather')


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Mission: Impossible II',
 'Born on the Fourth of July',
 'Public Enemies',
 'The Bridge on the River Kwai',
 'The Interpreter']

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Encode categorical features
df['Genre'] = df['Genre'].astype('category').cat.codes
df['Director'] = df['Director'].astype('category').cat.codes
df['Cast'] = df['Cast'].astype('category').cat.codes

# Target: Recommend based on rating threshold
df['Target'] = (df['IMDB Rating'] > 8.0).astype(int)

X = df[['Genre', 'Director', 'Cast', 'Duration', 'Votes']]
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict
predictions = model.predict(X_test)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [22]:
# Updated weighted recommendation function
def recommend_weighted(movie_name, top_n=5):
    # Calculate a weighted score
    df['Score'] = (
        0.4 * df['IMDB Rating'] + 
        0.3 * (df['Metascore'] / 100) + 
        0.2 * (df['Duration'] / df['Duration'].max()) + 
        0.1 * (df['Votes'] / df['Votes'].max())
    )
    
    # Sort movies by score and return top N
    return df.sort_values(by='Score', ascending=False)['Movie Name'].head(top_n).tolist()

# Example usage
recommended_movies = recommend_weighted('The Godfather', top_n=5)

print("Recommended Movies:")
print(recommended_movies)


Recommended Movies:
['The Godfather', 'The Lord of the Rings: The Return of the King', "Schindler's List", 'The Lord of the Rings: The Fellowship of the Ring', 'The Godfather Part II']


In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten

# Assume `user_movie_df` matrix exists with users and movies
num_users = len(df['Votes'].unique())
num_movies = len(df['Movie Name'].unique())

user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

user_embedding = Embedding(num_users, 50)(user_input)
movie_embedding = Embedding(num_movies, 50)(movie_input)

dot_product = Dot(axes=1)([Flatten()(user_embedding), Flatten()(movie_embedding)])
model = tf.keras.models.Model([user_input, movie_input], dot_product)
model.compile(optimizer='adam', loss='mean_squared_error')

# Training is similar to collaborative filtering but in a neural network context.


In [21]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Use previously calculated cosine similarity (content-based)
cosine_sim = cosine_similarity(combined_features, combined_features)

# Mock collaborative filtering scores (for demonstration purposes)
# You can replace this with the actual collaborative filtering similarity matrix if available.
cf_sim = cosine_similarity(combined_features, combined_features)

# Hybrid recommendation function
def recommend_hybrid(movie_name, alpha=0.5, num_recommendations=5):
    # Check if the movie exists
    if movie_name not in df['Movie Name'].values:
        print(f"'{movie_name}' not found in the dataset!")
        return []

    # Get the index of the movie
    movie_idx = df[df['Movie Name'] == movie_name].index[0]

    # Content-based recommendations
    content_similarity_scores = cosine_sim[movie_idx]

    # Collaborative filtering recommendations (mock example here)
    collaborative_similarity_scores = cf_sim[movie_idx]

    # Combine scores with weighting
    hybrid_scores = alpha * content_similarity_scores + (1 - alpha) * collaborative_similarity_scores

    # Sort scores in descending order and get the top N recommendations
    similar_indices = np.argsort(hybrid_scores)[::-1][1:num_recommendations + 1]
    recommended_movies = df.iloc[similar_indices]['Movie Name'].tolist()

    return recommended_movies

# Example usage
movie_to_recommend = 'The Godfather'
recommended_movies = recommend_hybrid(movie_to_recommend, alpha=0.5, num_recommendations=5)

print(f"Movies recommended for '{movie_to_recommend}':")
print(recommended_movies)

Movies recommended for 'The Godfather':
['The Godfather Part II', 'The Godfather Part III', 'Apocalypse Now', 'The Rainmaker', 'On the Waterfront']


In [26]:
# from surprise import SVD, Dataset, Reader
# from surprise.model_selection import train_test_split
# from surprise import accuracy

# # Prepare the data for collaborative filtering
# reader = Reader(rating_scale=(1, 10))  # Adjust based on the `Votes` scale
# data = Dataset.load_from_df(df[['Movie Name', 'Votes', 'IMDB Rating']], reader)

# # Train-test split
# trainset, testset = train_test_split(data, test_size=0.2)

# # Train an SVD model
# model = SVD()
# model.fit(trainset)

# # Test the model
# predictions = model.test(testset)
# accuracy.rmse(predictions)

# # Recommendation function
# def recommend_cf(movie_name, top_n=5):
#     movie_id = df[df['Movie Name'] == movie_name].index[0]
#     predictions = [model.predict(uid, movie_id) for uid in range(len(df))]
#     top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n]
#     recommended_movies = [df.iloc[p.iid]['Movie Name'] for p in top_predictions]
#     return recommended_movies

# # Example usage
# recommend_cf('The Rainmaker')
